<a href="https://colab.research.google.com/github/alessandrobessi/cuda-lab/blob/master/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -sf /usr/bin/gcc-5/usr/local/cuda/bin/gcc;
!ln -sf /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

50 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-01-16 15:40:13--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?MEUj1qfXjpxRKJghyXWTJ11gYSF_9FSRLEfAjx4FTj_DsroZqjRIrOHmY4KLkuhb4kZ1ZZfeH_x6mWkY2FV3qmLMvBvENQZTFBMV5eX4uLYYkxq6VPm7l5sz1Nejtr3paaViWfp-KTQuwNDCDvFyqj0rApoXwMwQ_aoyc27re3ptsTWi1v8j4SbGiDKE3F4-1O8KsJO6P918Xt1ghoNNi6tE0w [following]
--2019-01-16 15:40:13--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?MEUj1qfXjpxRKJghyXWTJ1

In [3]:
!/usr/local/cuda/bin/nvcc --version
!gcc --version
!g++ --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61
gcc (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

g++ (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
!pip install git+git://github.com/alessandrobessi/nvcc4jupyter.git

  Cloning git://github.com/alessandrobessi/nvcc4jupyter.git to /tmp/pip-req-build-2pwegbru
  Running setup.py bdist_wheel for NVCCPlugin ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-1s_tpjrx/wheels/af/b7/9c/ef07a4aaee4995dc4cf4740454641b9b99b6aad7aa2dab2ee2
Successfully built NVCCPlugin


In [5]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
%load_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [14]:
%%cu
#include <stdio.h>

int main() {
    printf("Hello world\n");
    return 0;
}

'Hello world\n'